# Fully connected NN

In this script, we are implementing a fully connected, 3-layers NN with the following layers:

* __DropOut__ for regularization
* __BatchNormalization__ for effective optimization
* __ReLU__ activation function
* __SoftMax/SVM__ as loss function
* __RMSProp/Adam__ as optimizer

In [1]:
from __future__ import print_function

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
### load sample of the data and split it to training and testing data
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

batch_1 = unpickle('../data/cifar-10-batches-py/data_batch_1')
data = batch_1['data']
labels = batch_1['labels']

train_data = data[:7000,:]
test_data = data[7000:,:]
y_train = labels[:7000]
y_test = labels[7000:]

print("Train data shape: {}".format(train_data.shape))
print("Test data shape: {}".format(test_data.shape))




Train data shape: (7000, 3072)
Test data shape: (3000, 3072)


In [25]:
### Normalize data and get batches of tensors
means = train_data.mean(axis=0)
stds = train_data.std(axis=0)

X_train = (train_data - means)/stds
X_test = (test_data - means)/stds

train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_loader = iter(torch.utils.data.DataLoader(train_dataset, batch_size=8))

test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
test_loader = iter(torch.utils.data.DataLoader(test_dataset, batch_size=8))

In [56]:
### define fully-connected NN
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, layers_sizes=[3072, 1024, 516, 10]):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(in_features=layers_sizes[0], out_features=layers_sizes[1])
        self.fc2 = nn.Linear(in_features=layers_sizes[1], out_features=layers_sizes[2])
        self.fc3 = nn.Linear(in_features=layers_sizes[2], out_features=10)
        self.bn = nn.BatchNorm1d(num_features=layers_sizes[1])
        
    def forward(self, x):
        x = F.dropout(F.relu(self.fc1(x)), p=0.2)
        x = self.bn(x)
        x = F.dropout(F.relu(self.fc2(x)), p=0.2)
        y_pred = self.fc3(x)
        return y_pred
        
net = Net()

In [57]:
### define loss criteria and optimizer
import torch.optim as optim

cross_entropy = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [66]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs).type(torch.FloatTensor), Variable(labels).type(torch.LongTensor)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [ ]:
### Make predictions